# Parameters

In [ ]:
from model_unet import Unet

params={    
    'model_func' : Unet,
    'model_params' : {
        'img_height' : 256,
        'img_width'  : 256,
        'nclasses'   : 8
    },
    'gen_params' : {'batch_size':4,
                    'dim':(256,256),
                    },
    'metrics' : [
            # dice_coeff,
            'accuracy'],
    'train_step_per_epoch' : None, #sample/batch_size
    'val_step_per_epoch' : None,
    'num_epochs' : 8,
    'training_description' : 'cityscape segmentation experiments',
    'model_name' : 'Unet',
    }


# Run



In [ ]:
from mlflow_tools import cityscape_experiment,start_local_experiment

start_local_experiment()

In [ ]:

cityscape_experiment(**params)